In [1]:
import numpy as np

In [2]:
def vector_add(a, b):
    return a + b

def vector_sub(a, b):
    return a - b

def scalar_mult(scalar, vector):
    return scalar * vector

In [3]:
def infnorm(a):
    return np.max(np.abs(a))

In [4]:
mu = 0.012277471  # Пример значения, должны быть определены в соответствии с задачей
mus = 1 - mu

def F(x, y1, y2, y2s):
    d1 = ((y1 + mu)**2 + y2**2)**1.5
    d2 = ((y1 - mus)**2 + y2**2)**1.5
    return y1 + 2 * y2s - mus * (y1 + mu) / d1 - mu * (y1 - mus) / d2

def G(x, y1, y2, y1s):
    d1 = ((y1 + mu)**2 + y2**2)**1.5
    d2 = ((y1 - mus)**2 + y2**2)**1.5
    return y2 - 2 * y1s - mus * y2 / d1 - mu * y2 / d2

In [5]:
def olem42_step(f, g, x0, y0, h, tol, x_fin, A11, A12, A21, A22, B12, B21, C1, C2, D1, Q1, Di1, Qi1, D2, Q2, Di2, Qi2):
    K1 = np.zeros(4)
    K2 = np.zeros(4)
    K1[0] = f(x0, y0[0], y0[2], y0[3])
    K2[0] = g(x0, y0[0], y0[2], y0[1])
    
    for i in range(1, 3):
        s2x_k1 = A11[i, :i] @ K1[:i]
        s2y_k1 = A12[i, :i] @ K2[:i]
        s2x_k2 = A21[i, :i] @ K1[:i]
        s2y_k2 = A22[i, :i] @ K2[:i]
        s2xs = B21[i, :i] @ K1[:i]
        s2ys = B12[i, :i] @ K2[:i]
        
        K1[i] = f(x0 + C1[i] * h, y0[0] + C1[i] * h * y0[1] + h * h * s2x_k1,
                  y0[2] + C1[i] * h * y0[3] + h * h * s2y_k1, y0[3] + h * s2ys)
        K2[i] = g(x0 + C2[i] * h, y0[0] + C2[i] * h * y0[1] + h * h * s2x_k2,
                  y0[2] + C2[i] * h * y0[3] + h * h * s2y_k2, y0[1] + h * s2xs)
    
    Y = y0.copy()
    Y[0] += h * y0[1]
    Y[2] += h * y0[3]
    Yi = Y.copy()
    
    Y[:4] += h * h * D1 @ K1 + h * Q1 @ K1
    Yi[:4] += h * h * Di1 @ K1 + h * Qi1 @ K1
    
    te = infnorm(Yi - Y)
    if te <= tol:
        y0 = Y
        x0 += h
        step_made = True
    else:
        step_made = False
    
    h = min(0.9 * h * (tol / te)**(1/3), x_fin - x0)
    return x0, y0, h, step_made

In [6]:
import numpy as np

# Параметры модели
mu = 0.012277471
mus = 1 - mu

# Определение функций F и G
def F(x, y1, y2, y2s):
    d1 = ((y1 + mu)**2 + y2**2)**1.5
    d2 = ((y1 - mus)**2 + y2**2)**1.5
    return y1 + 2 * y2s - mus * (y1 + mu) / d1 - mu * (y1 - mus) / d2

def G(x, y1, y2, y1s):
    d1 = ((y1 + mu)**2 + y2**2)**1.5
    d2 = ((y1 - mus)**2 + y2**2)**1.5
    return y2 - 2 * y1s - mus * y2 / d1 - mu * y2 / d2

# Бесконечная норма
def infnorm(a):
    return np.max(np.abs(a))

# Инициализация матриц и векторов
C1 = np.array([0, 1.0 / 3, 5.0 / 6, 1])  # Последнее значение добавлено для комплектности
C2 = np.array([1.0 / 6, 2.0 / 3, 1, 0])  # Последнее значение добавлено для комплектности

D1 = np.array([1.0 / 10, 1.0 / 3, 1.0 / 15, 0])  # Добавлены нули для размерности
Q1 = np.array([1.0 / 10, 1.0 / 2, 2.0 / 5, 0])   # Добавлены нули для размерности
Di1 = np.array([2.0 / 5, -1.0 / 6, 4.0 / 15, 0])  # Добавлены нули для размерности
Qi1 = np.array([2.0 / 5, 0, 3.0 / 5, 0])          # Добавлены нули для размерности

D2 = np.array([1.0 / 3, 1.0 / 6, 0, 0])           # Добавлены нули для размерности
Q2 = np.array([2.0 / 5, 1.0 / 2, 1.0 / 10, 0])    # Добавлены нули для размерности
Di2 = np.array([1.0 / 4, 1.0 / 4, 0, 0])          # Добавлены нули для размерности
Qi2 = np.array([3.0 / 5, 0, 2.0 / 5, 0])          # Добавлены нули для размерности

# Начальные условия
y0 = np.array([0.994, 0, 0, -2.0015851063790834])  # Значения из вашего кода C++
t0 = 0
tfin = 17.065216556015796
tol = 1e-5  # Предположим, что это допустимая погрешность

# Предположим начальный шаг
h = 0.01  # Начальный шаг

# Матрицы A и B
# Для упрощения примера я укажу только нулевые или очень маленькие матрицы
# Реальные матрицы должны быть определены согласно вашим требованиям
A11 = np.zeros((4, 4))
A12 = np.zeros((4, 4))
A21 = np.zeros((4, 4))
A22 = np.zeros((4, 4))
B12 = np.zeros((4, 4))
B21 = np.zeros((4, 4))

# Запуск интеграции
step_made = False
while t0 < tfin and not step_made:
    t0, y0, h, step_made = olem42_step(F, G, t0, y0, h, tol, tfin, A11, A12, A21, A22, B12, B21, C1, C2, D1, Q1, Di1, Qi1, D2, Q2, Di2, Qi2)
    print(f"Time: {t0}, State: {y0}, Step size: {h}")

Time: 0, State: [ 0.994       0.          0.         -2.00158511], Step size: 0.00025023211555009214
Time: 0, State: [ 0.994       0.          0.         -2.00158511], Step size: 0.00012290703583042492
Time: 0.00012290703583042492, State: [ 0.95524453 -0.03875547 -0.03900148 -2.04034058], Step size: 0.00012264493670422554


In [9]:
import numpy as np

mu = 0.012277471
mus = 1 - mu

def F(x, y1, y2, y2s):
    d1 = ((y1 + mu)**2 + y2**2)**1.5
    d2 = ((y1 - mus)**2 + y2**2)**1.5
    return y1 + 2 * y2s - mus * (y1 + mu) / d1 - mu * (y1 - mus) / d2

def G(x, y1, y2, y1s):
    d1 = ((y1 + mu)**2 + y2**2)**1.5
    d2 = ((y1 - mus)**2 + y2**2)**1.5
    return y2 - 2 * y1s - mus * y2 / d1 - mu * y2 / d2

def infnorm(a):
    return np.max(np.abs(a))

# Определение всех матриц и векторов
C1 = np.array([0, 1.0 / 3, 5.0 / 6])
C2 = np.array([1.0 / 6, 2.0 / 3, 1])

D1 = np.array([1.0 / 10, 1.0 / 3, 1.0 / 15, 0])  # Добавим недостающий элемент
Q1 = np.array([1.0 / 10, 1.0 / 2, 2.0 / 5, 0])   # Добавим недостающий элемент
Di1 = np.array([2.0 / 5, -1.0 / 6, 4.0 / 15, 0]) # Добавим недостающий элемент
Qi1 = np.array([2.0 / 5, 0, 3.0 / 5, 0])         # Добавим недостающий элемент

D2 = np.array([1.0 / 3, 1.0 / 6, 0, 0])          # Уже имеет нужный размер
Q2 = np.array([2.0 / 5, 1.0 / 2, 1.0 / 10, 0])   # Добавим недостающий элемент
Di2 = np.array([1.0 / 4, 1.0 / 4, 0, 0])         # Уже имеет нужный размер
Qi2 = np.array([3.0 / 5, 0, 2.0 / 5, 0])         # Уже имеет нужный размер

# Исправленная инициализация матриц с одинаковым количеством элементов в каждой строке
A11 = np.array([
    [0, 0, 0],
    [1.0 / 18, 0, 0],
    [5.0 / 144, 5.0 / 16, 0]
])

A12 = np.array([
    [0, 0, 0],
    [1.0 / 18, 0, 0],
    [5.0 / 18, 5.0 / 72, 0]
])

A21 = np.array([
    [1.0 / 36, 0, 0],
    [5.0 / 72, 1.0 / 8, 0],
    [-1.0 / 24, 5.0 / 8, 0]
])

A22 = np.array([
    [0, 0, 0],
    [1.0 / 4, 0, 0],
    [5.0 / 36, 5.0 / 18, 0]
])

B12 = np.array([
    [0, 0, 0],
    [1.0 / 3, 0, 0],
    [5.0 / 12, 5.0 / 12, 0]
])

B21 = np.array([
    [1.0 / 6, 0, 0],
    [-1.0 / 12, 3.0 / 4, 0],
    [3.0 / 4, -5.0 / 12, 2.0 / 3]
])

y0 = np.array([0.994, 0, 0, -2.0015851063790834])  # Значения из вашего кода C++
t0 = 0
tfin = 17.065216556015796
tol = 1e-5  # Предположим, что это допустимая погрешность

# Предположим начальный шаг
h = 0.01  # Начальный шаг

# Функция для выполнения одного шага
def olem42_step(f, g, x0, y0, h, tol, x_fin, A11, A12, A21, A22, B12, B21, C1, C2, D1, Q1, Di1, Qi1, D2, Q2, Di2, Qi2):
    K1 = np.zeros(4)
    K2 = np.zeros(4)
    K1[0] = f(x0, y0[0], y0[2], y0[3])
    K2[0] = g(x0, y0[0], y0[2], y0[1])
    
    for i in range(1, 3):
        s2x_k1 = np.dot(A11[i, :i], K1[:i])
        s2y_k1 = np.dot(A12[i, :i], K2[:i])
        s2x_k2 = np.dot(A21[i, :i], K1[:i])
        s2y_k2 = np.dot(A22[i, :i], K2[:i])
        s2xs = np.dot(B21[i, :i], K1[:i])
        s2ys = np.dot(B12[i, :i], K2[:i])
        
        K1[i] = f(x0 + C1[i] * h, y0[0] + C1[i] * h * y0[1] + h * h * s2x_k1,
                  y0[2] + C1[i] * h * y0[3] + h * h * s2y_k1, y0[3] + h * s2ys)
        K2[i] = g(x0 + C2[i] * h, y0[0] + C2[i] * h * y0[1] + h * h * s2x_k2,
                  y0[2] + C2[i] * h * y0[3] + h * h * s2y_k2, y0[1] + h * s2xs)
    
    Y = y0.copy()
    Y[0] += h * y0[1]
    Y[2] += h * y0[3]
    Yi = Y.copy()
    
    Y[:4] += h * h * D1 @ K1 + h * Q1 @ K1
    Yi[:4] += h * h * Di1 @ K1 + h * Qi1 @ K1
    
    te = infnorm(Yi - Y)
    if te <= tol:
        y0 = Y
        x0 += h
        step_made = True
    else:
        step_made = False
    
    h = min(0.9 * h * (tol / te)**(1/3), x_fin - x0)
    return x0, y0, h, step_made

# Запуск интеграции
while t0 < tfin:
    t0, y0, h, step_made = olem42_step(F, G, t0, y0, h, tol, tfin, A11, A12, A21, A22, B12, B21, C1, C2, D1, Q1, Di1, Qi1, D2, Q2, Di2, Qi2)
    print(f"Time: {t0}, State: {y0}, Step size: {h}")
    if not step_made:
        break

Time: 0, State: [ 0.994       0.          0.         -2.00158511], Step size: 0.000259448970929731
